# Actor-Critic


<img src="ac_scheme.png">

«Критик» оценивает функцию полезности. Это может быть полезность действия ($Q$) или полезность состояния ($V$).

«Актор» обновляет распределение стратегии в направлении, предложенном Критиком.


Псевдокод для $Q$ версии алгоритма приведен ниже:
<img src="alg_ac.png" style="width: 700px">


### Задание 1. Заполните пропуски для $V$ версии алгоритма Actor-Critic

In [21]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

learning_rate = 0.0002
gamma = 0.98

In [23]:
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.data = []

        # создаем сеть с двумя головами для pi и V (подсказка - смотрим на методы pi и v)
        #~~~~~~~~ Ваш код здесь ~~~~~~~~~~~        
        raise NotImplementedError        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        

        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def pi(self, x, softmax_dim=0):
        x = F.relu(self.fc1(x))
        x = self.fc_pi(x)
        prob = F.softmax(x, dim=softmax_dim)
        return prob

    def v(self, x):
        x = F.relu(self.fc1(x))
        v = self.fc_v(x)
        return v

    def put_data(self, transition):
        self.data.append(transition)

    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, done_lst = [], [], [], [], []

        for transition in self.data:
            # собираем батч из сохранненых кортежей
            # не забывайте сделать клиппинг вознаграждений!
            #~~~~~~~~ Ваш код здесь ~~~~~~~~~~~            
            raise NotImplementedError            
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            
            # не забываем про окончание эпизода
            done_mask = 0.0 if done else 1.0
            done_lst.append([done_mask])

        s_batch, a_batch, r_batch, s_prime_batch, done_batch = torch.tensor(s_lst, dtype=torch.float), torch.tensor(
            a_lst), \
                                                               torch.tensor(r_lst, dtype=torch.float), torch.tensor(
            s_prime_lst, dtype=torch.float), \
                                                               torch.tensor(done_lst, dtype=torch.float)
        self.data = []
        return s_batch, a_batch, r_batch, s_prime_batch, done_batch

    def train_net(self):
        # получаем батч
        # s, a, r, s_prime, done =
        #~~~~~~~~ Ваш код здесь ~~~~~~~~~~~        
        raise NotImplementedError        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        

        # считаем td target(r + gamma ...) и td ошибку (td_target - v)
        # td_target = 
        # delta = 
        #~~~~~~~~ Ваш код здесь ~~~~~~~~~~~        
        raise NotImplementedError        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        

        # получаем распределение
        pi = self.pi(s, softmax_dim=1)
        pi_a = pi.gather(1, a)

        # считаем td_loss (подсказка можно воспользоваться smooth_l1_loss) (не забыаем про лишние градиенты!)
        # td_loss = 
        td_loss = F.smooth_l1_loss(self.v(s), td_target.detach())
        # считаем loss актора (-pi_a) * delta (не забыаем про лишние градиенты!)
        # a_loss =  
        a_loss = -torch.log(pi_a) * delta.detach()

        # вызываем backward и делаем шаг оптимизатора для каждого лосса
        #~~~~~~~~ Ваш код здесь ~~~~~~~~~~~        
        raise NotImplementedError        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        


### Задание 2. Подбираем гиперпараметры и строим графики. Как влияет клиппинг вознаграждений на итоговую сходимость?

In [24]:
def run():
    env = gym.make('CartPole-v1')
    model = ActorCritic()
    n_rollout = 5
    print_interval = 20
    score = 0.0

    for n_epi in range(1000):
        done = False
        s = env.reset()
        while not done:
            for t in range(n_rollout):
                prob = model.pi(torch.from_numpy(s).float())
                m = Categorical(prob)
                a = m.sample().item()
                s_prime, r, done, info = env.step(a)
                model.put_data((s, a, r, s_prime, done))

                s = s_prime
                score += r

                if done:
                    break

            model.train_net()

        if n_epi % print_interval == 0 and n_epi != 0:
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, score / print_interval))
            score = 0.0
    env.close()


run()